# Create codelist for daytime sleepiness project

Get a list of all VMPs (Virtual Medicinal Products) and AMPs (Actual Medicinal Products) from a given list of ingredients (VTMs = Virtual Therapeutic Moieties)

In [28]:
# set name of codelist for exporting file
codelist_name = "narcolepsy"

# import or paste list of vtms (Virtual Therapeutic Moieties / Ingredients) by name
names = ['Sodium Oxybate', 'Pitolisant', 'Solriamfetol', 'Modafinol', 'Dexamfetamine', 'Methylphenidate']

from ebmdatalab import bq
from termcolor import colored
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Select products from DMD containing chosen ingredients

In [44]:
# convert list to tuple for use in SQL query
names_tuple = ["%Methoxypsoralen%"]

sql = f'''
SELECT *
FROM (SELECT 'vmp' AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd 
FROM dmd.vmp
INNER JOIN dmd.vpi AS vpi ON vmp.id = vpi.vmp 
INNER JOIN dmd.ing as ing ON ing.id = vpi.ing 
LEFT JOIN dmd.ddd on vmp.id = ddd.vpid
ORDER BY type, nm) 

WHERE (`ingredient` LIKE ('{names_tuple}'))
'''

meds = bq.cached_read(sql, csv_path=os.path.join('..','data',f'meds_{codelist_name}.csv'))

meds.count()

meds

GenericGBQException: Reason: 400 Syntax error: Illegal input character "%" at [10:30]

(job ID: 7654cc28-bfff-4dc8-b084-0f4fc0c08b88)

                             -----Query Job SQL Follows-----                             

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:SELECT *
   3:FROM (SELECT 'vmp' AS type, vmp.id, bnf_code, vmp.nm, ing.nm AS ingredient, ddd.ddd 
   4:FROM dmd.vmp
   5:INNER JOIN dmd.vpi AS vpi ON vmp.id = vpi.vmp 
   6:INNER JOIN dmd.ing as ing ON ing.id = vpi.ing 
   7:LEFT JOIN dmd.ddd on vmp.id = ddd.vpid
   8:ORDER BY type, nm) 
   9:
  10:WHERE (`ingredient` LIKE ('['%Methoxypsoralen%', 'Dexamfetamine']'))
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |

## Check if any ingredients were not found

In [14]:
ings = list(meds.ingredient.drop_duplicates())
names.sort()
ings.sort()

print_ing_selected = "Selected ingredients: " + str(names)
print_ing_found = "Ingredients found: " + str(ings)

ings_missing = [i for i in names if i not in ings]
print_ing_missing = "Ingredients NOT found:" + str(ings_missing)

print(print_ing_selected)
print(colored(print_ing_found, 'green'))
print(colored(print_ing_missing, 'red'))

Selected ingredients: ['Dexamfetamine', 'Methylphenidate', 'Modafinol', 'Pitolisant', 'Sodium Oxybate', 'Solriamfetol']
Ingredients found: ['Dexamfetamine']
Ingredients NOT found:['Methylphenidate', 'Modafinol', 'Pitolisant', 'Sodium Oxybate', 'Solriamfetol']


In [15]:
# check how many products have DDDs
print(meds[["ddd", "id"]].count())

ddd    0
id     7
dtype: int64


## Show table

In [16]:
meds

,type,id,bnf_code,nm,ingredient,ddd
0,vmp,15852011000001105,0404000L0AAAFAF,Amfetamine 10mg / Dexamfetamine 10mg modified-release capsules,Dexamfetamine,NaN
1,vmp,15852111000001106,0404000L0AABFBF,Amfetamine 12.5mg / Dexamfetamine 12.5mg modified-release capsules,Dexamfetamine,NaN
2,vmp,15852211000001100,0404000L0AAAYAY,Amfetamine 15mg / Dexamfetamine 15mg modified-release capsules,Dexamfetamine,NaN
3,vmp,15852311000001108,0404000L0AABDBD,Amfetamine 2.5mg / Dexamfetamine 2.5mg modified-release capsules,Dexamfetamine,NaN
4,vmp,15852411000001101,0404000L0AABHBH,Amfetamine 5mg / Dexamfetamine 5mg modified-release capsules,Dexamfetamine,NaN
5,vmp,15852511000001102,0404000L0AAAVAV,Amfetamine 5mg / Dexamfetamine 5mg tablets,Dexamfetamine,NaN
6,vmp,15852611000001103,0404000L0AABEBE,Amfetamine 7.5mg / Dexamfetamine 7.5mg modified-release capsules,Dexamfetamine,NaN
